In [ ]:
def transform_train(size=224, aug=None):
    if aug=="basic":
        print("basic aug")
        return transforms.Compose([
            transforms.RandomResizedCrop(size),
            transforms.RandomAffine(45),
            transforms.RandomRotation(90),
            transforms.RandomVerticalFlip(),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
    else:
        return transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
        
        
def transform_evaluation(size=224):
    return transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".png", ".jpg", ".jpeg"])


class DatasetFromFolder(torch.utils.data.Dataset):
    def __init__(self, image_dir, transform=None):
        super(DatasetFromFolder, self).__init__()
        self.image_filenames = [join(image_dir, x) for x in listdir(image_dir) if is_image_file(x)]

        self.transform = transform

    def __getitem__(self, index):
        filename = self.image_filenames[index]
        input = Image.open(filename)
        #label = basename(filename).split('_', 1)[0]
        if self.transform:
            input = self.transform(input)
        label = basename(filename).split('_')[0:2]
        if label[0].isdigit():
            label = label[0]
        else:
            label = label[1]
        #print("filename, label", filename, label)
        return input, int(label)

    def __len__(self):
        return len(self.image_filenames)
    
    
def get_training_set(datadir, aug=None):
    train_dir = join(datadir, "Food-11/training")
    
    return DatasetFromFolder(train_dir, transform=transform_train(IMAGE_SIZE, aug))


def get_validation_set(datadir):
    validation_dir = join(datadir, "Food-11/validation")
    
    return DatasetFromFolder(validation_dir, transform=transform_train(IMAGE_SIZE))


def get_evaluation_set(datadir, aug=False):
    if aug:
        evaluation_dir = join(datadir, "food_test2/evaluation")
    else:
        evaluation_dir = join(datadir, "Food-11/evaluation")
    
    return DatasetFromFolder(evaluation_dir, transform=transform_train(IMAGE_SIZE))



In [ ]:
def save_checkpoint(state, is_best, filename=None):
    
    if filename == None:
        #filename = './ckpt/ckpt' + '-epoch(' + str(state['epoch']) + ')-prec1(' + str("%.3f" % state['best_prec1']) + ').pth.tar'
        filename = './ckpt/ckpt' + '-epoch(' + str(state['epoch']) + ').pth.tar'

    if state['epoch'] % 5 == 0:
        torch.save(state, filename)
    
    if is_best:
        torch.save(state, './ckpt/model_best.pth.tar')
    
        

def load_best_checkpoint(filename='./ckpt/model_best.pth.tar'):
    if isfile(filename):
        print('=> loading best checkpoint')
        checkpoint = torch.load(filename)
        epoch = checkpoint['epoch']
        best_accu = checkpoint['best_accu']
        train_accu, val_accu = checkpoint['accuracy']
        train_loss, val_loss = checkpoint['loss']
        
    else:
        checkpoint = None
        epoch = 0
        best_prec1 = 0
        train_accu, val_accu = [], []
        train_loss, val_loss = [], []
        
    return checkpoint, epoch, best_accu, train_accu, val_accu, train_loss, val_loss
    

            
            

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def adjust_learning_rate(optimizer, epoch):
    # Sets the learning rate to the initial LR decayed by 10 every 30 epochs
    #lr = LR * (0.1 ** (epoch // 30))
    # Sets the learning rate to the initial LR decayed by 2 every 10 epochs
    lr = LR * (0.5 ** (epoch // 10))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    print('Learning rate is {:.5f}'.format(lr))
        
        
def accuracy(output, target, topk=(1,)):
    # Computes the precision@k for the specified values of k
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)
        
        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


In [ ]:
# Plot Accuracy & Loss curve
def draw_accuracy(accuracy, loss, accuracy1 = None, loss1 = None):
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(8,3), dpi=80)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy (%)')
    ax1.set_title('Accuracy')
    ax1.grid(b=True)

    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.set_title('Loss')
    ax2.grid(b=True)

    ax1.plot(accuracy, label='Traing Accuracy')
    ax2.plot(loss, label='Traing Loss')
    if accuracy1 != None and loss1 != None:
        ax1.plot(accuracy1, label='Validation Accuracy')
        plt.legend()
        ax2.plot(loss1, label='Validation Loss')
        plt.legend()
        
    plt.tight_layout()

#draw_accuracy(accuvalue, lossvalue)

def draw_dual_accuracy(accuracy, loss, accuracy1 = None, loss1 = None):
    fig1, ax1 = plt.subplots(1,1,figsize=(4,3), dpi=100)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy (%)')
    ax1.set_title('Accuracy')
    ax1.grid(b=True)
    ax1.plot(accuracy[0], label='Train')
    ax1.plot(accuracy[1], label='Train Aug.')
    ax1.plot(accuracy1[0], label='Val.')
    ax1.plot(accuracy1[1], label='Val. Aug.')
    plt.legend()
        
    fig2, ax2 = plt.subplots(1,1,figsize=(4,3), dpi=100)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.set_title('Loss')
    ax2.grid(b=True)
    ax2.plot(loss[0], label='Train')
    ax2.plot(loss[1], label='Train Aug.')
    ax2.plot(loss1[0], label='Val.')
    ax2.plot(loss1[1], label='Val. Aug.')
    plt.legend()
        
    plt.tight_layout()

def draw_dual_accuracy0(accuracy, loss, accuracy1 = None, loss1 = None):
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(8,3), dpi=80)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy (%)')
    ax1.set_title('Accuracy')
    ax1.grid(b=True)

    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.set_title('Loss')
    ax2.grid(b=True)

    ax1.plot(accuracy[0], label='Traing Accuracy')
    ax1.plot(accuracy[1], label='Traing Augmentation Accuracy')
    ax2.plot(loss[0], label='Traing Loss')
    ax2.plot(loss[1], label='Traing Augmentation Loss')
    if accuracy1 != None and loss1 != None:
        ax1.plot(accuracy1[0], label='Validation Accuracy')
        ax1.plot(accuracy1[1], label='Validation Augmentation Accuracy')
        ax2.plot(loss1[0], label='Validation Loss')
        ax2.plot(loss1[1], label='Validation Augmentation Loss')
        plt.legend()
        
    plt.tight_layout()


# Plot Weight & Bias distribution
def draw_distribution(ax, data, title='Weights Distribution', xlabel='Value', ylabel='Frequency', bins='auto'):
    
    d = data.numpy().reshape(-1)
    d_mean = np.mean(d, axis=0)
    d_std  = np.std(d, axis=0, ddof=1)

    n, bins, patches = ax.hist(x=d, bins='auto', color='#0504aa', alpha=0.7, rwidth=0.85)
    
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    
    ax.grid()
    
    maxfreq = n.max()
    ymax = np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10
    
    ax.set_ylim(top=ymax)#, bottom=0)
    
    tyaxis = ymax * 0.9
    
    maxvalue = max(abs(bins.max()), abs(bins.min()))
    minvalue = -maxvalue
    ax.set_xlim(left=minvalue, right=maxvalue)
    
    txaxis = minvalue + (maxvalue - minvalue) * 0.05
    
    ax.text(txaxis, tyaxis, r'$\mu={:6.3f}$'.format(d_mean))
    ax.text(txaxis, tyaxis*0.9, r'$\sigma={:6.3f}$'.format(d_std))
    
            
def show_sta(models, layer=0):
    
    for m in models:
        #print(layer, m)
        if isinstance(m, nn.Conv2d):
            #print("==> L{:d} Conv2d:".format(layer), m)
            fig, (ax1, ax2) = plt.subplots(1,2,figsize=(8,3), dpi=80)

            draw_distribution(ax1, m.weight.data.cpu(), 'Conv2d Weights Distribution')

            try:
                draw_distribution(ax2, m.bias.data.cpu(), 'Conv2d Bias Distribution')
            except AttributeError:
                draw_distribution(ax2, torch.zeros(5), 'Conv2d Bias NOT exist')

        elif isinstance(m, nn.Linear):
            #print("==> L{:d} Linear:".format(layer), m)
            fig, (ax1, ax2) = plt.subplots(1,2,figsize=(8,3), dpi=80)

            draw_distribution(ax1, m.weight.data.cpu(), 'Linear Weights Distribution')

            try:
                draw_distribution(ax2, m.bias.data.cpu(), 'Linear Bias Distribution')
            except AttributeError:
                draw_distribution(ax2, torch.zeros(5), 'Linear Bias NOT exist')


        elif isinstance(m, nn.Sequential):
            show_sta(m, layer+1)

        elif isinstance(m, BasicBlock):
            show_sta(m.children(), layer+1)
            

            
def show_distribution(model):
    params=model.state_dict()
    even = True
    plt.subplots_adjust(wspace =1, hspace =1)
    
    for k,v in params.items():
        
        if k.split('.')[-1] == 'weight' or k.split('.')[-1] == 'bias':
            
            if even:
                fig, (ax1, ax2) = plt.subplots(1,2,figsize=(9,3), dpi=80)
                ax = ax1            
            else:
                ax = ax2

            title = k + '.Distribution'
            draw_distribution(ax, v.data.cpu(), title)
            even = not even
    

        
#show_sta(net.children())

In [1]:
def show_class_accuracy(eva_accuracy, eva_loss, accu_class, classes):
    y_pos = np.arange(len(classes))

    fig, ax1 = plt.subplots(1,1)
    plt.rcdefaults()
            
    plt.barh(y_pos, accu_class, align='center', alpha=0.4)
    plt.yticks(y_pos, classes)
    plt.xlabel('Accuracy')

    plt.title('Total accuracy is {:.2f}% and loss is {:.5f}'.format(eva_accuracy, eva_loss))

    for xx, yy in zip(accu_class, y_pos):
        plt.text(xx*0.95, yy, '{:.3f}'.format(xx), ha='center')
        
        
def show_table_result(classes, result):
    
    fig, ax = plt.subplots(figsize=(16,3), dpi=80)
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_visible(False)
    
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.table(cellText=result, rowLabels=classes[::-1], colLabels=classes[::-1], loc='best')
    fig.tight_layout()
    
